In [ ]:
import pymecht as pmt
import numpy as np

## Definition of the material model

In [ ]:
mat1 = pmt.MatModel('yeoh')
print(mat1.parameters) # Returns parameters as a dictionary

In [ ]:
print(mat1.parameters['c1_0'])
type(mat1.parameters['c1_0'])

Combine two materials

In [ ]:
mat2 = pmt.GOH()
model = pmt.MatModel('Yeoh',mat2)

# model = pmt.MatModel(mat1,mat2) #Another way to combine two materials 

Define fiber directions

In [ ]:
# model.models[0].fiber_dirs = [np.array([0,cos(0.1),sin(0.1)]),np.array([0,cos(-0.1),sin(-0.1)])] #Attributes fiber directions to the first material 
model.models[1].fiber_dirs = [np.array([0,np.cos(0.1),np.sin(0.1)]),np.array([0,np.cos(-0.1),np.sin(-0.1)])] #Attributes fiber directions to the second material 

In [ ]:
model.parameters 
#parameters with index 0 correspond to the first material in model #parameters with index 1 correspond to the second

## Sample & parameter definition

In [ ]:
sample = pmt.UniformAxisymmetricTubeInflationExtension(model, force_measure='pressure')
print(sample)
params = sample.parameters
params #Additional sample parameters are added to the dictionnary

Set parameters to a specific value

In [ ]:
params.set('Ri', 9) 
params.set('thick', 2)
params.set('omega', 0)
params.set('L0', 1)
params.set('lambdaZ', 1)
params.set('c1_0', 28)
params.set('c2_0', 21)
params.set('c3_0', 8)
params.set('c4_0', 1)
params.set('k1_1',5.)
params.set('k2_1',15)
params.set('k3_1',0.1)
params

Fix parameters

In [ ]:
params.fix('Ri')
params.fix('thick')
params.fix('omega')
params.fix('L0')
params.fix('lambdaZ')
params

In [ ]:
sample #Recaps the material model and sample created

## Run the test

Force control

In [ ]:
from matplotlib import pyplot as plt
inp = np.linspace(1,20,100)
plt.plot(inp,sample.force_controlled(inp,params)) 
plt.ylabel('Internal radius in mm')
plt.xlabel('Internal pressure in kPa')
plt.title('Force control plot')
plt.show()

Displacement control

In [ ]:
from matplotlib import pyplot as plt
inp = np.linspace(9,12,100)
plt.plot(inp,sample.disp_controlled(inp,params)) 
plt.ylabel('Internal pressure in kPa')
plt.xlabel('Internal radius in mm')
plt.title('Displacement control plot')
plt.show()

## Stress across the thickness for a given radius

In [ ]:
xi,stress = sample.cauchy_stress([10], params)

In [ ]:
def von_mises(sigma_list):
    return [sqrt(3./2.)*np.linalg.norm(sigma-np.trace(sigma)/3.*np.eye(3)) for sigma in sigma_list]

plt.plot(xi,von_mises(stress))
plt.xlabel('Normalized thickness')
plt.ylabel('von-Mises stress')
plt.show()

## Least square fitting

In [ ]:
applied_radius = np.linspace(9,12,20)
measured_pressure = np.linspace(0,10,20)
def sim_func(param):
    return sample.disp_controlled(applied_radius,param)

param_fitter = pmt.ParamFitter(sim_func,measured_pressure,params)

In [ ]:
res = param_fitter.fit()

In [ ]:
print(params)

In [ ]:

plt.plot(applied_radius,sample.disp_controlled(applied_radius,params)) 
plt.plot(applied_radius,measured_pressure,'o')
plt.ylabel('Internal pressure in kPa')
plt.xlabel('Internal radius in mm')
plt.title('Displacement control plot')
plt.show()
